# Watch Me Code 3: Mapping with Folium

Folium is a Python wrapper library for the OpenStreetMaps api. It allows you to place data on a map in a variety of ways.


In [3]:
! pip install folium

     |████████████████████████████████| 92kB 3.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import folium
import pandas as pd
import random 

In [5]:
# we need to center the map in the middle of the US. I googled for the location.
CENTER_US = (39.8333333,-98.585522)
london = (51.5074, -0.1278)
map = folium.Map(location=CENTER_US, zoom_start=4)
map

## Map Pins

In [6]:
# read in a data file of IP address to locations.
data = pd.read_csv('https://raw.githubusercontent.com/mafudge/datasets/master/clickstream/ip_lookup.csv')
data.sample(5)

,IP,Country,State,City,ApproxLat,ApproxLng
2,98.29.25.44,USA,OH,Cleveland,41.499320,-81.694361
11,54.114.107.209,USA,NJ,Jersey City,40.728157,-74.077642
4,155.100.169.152,USA,UT,Salt Lake City,40.760779,-111.891047
7,74.111.6.173,USA,VA,Arlington,38.879970,-77.106770
6,70.209.14.54,USA,FL,Tampa,27.950575,-82.457178


In [7]:
# Let's place each location on the map
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State'])
                          )
    map.add_children(marker)
map

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [8]:
# Same thing with a different icon and colors. Icons come from http://fontawesome.io/icons/ but its an older version.
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
            'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
            'darkpurple', 'pink', 'lightblue', 'lightgreen',
            'gray', 'black', 'lightgray']
for row in data.to_records():
    pos = (row['ApproxLat'],row['ApproxLng'])
    marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (row['City'],row['State']),
                    icon = folium.Icon(color = random.choice(colors), icon='user')
                          )
    map.add_children(marker)
map

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: Method `add_children` is deprecated. Please use `add_child` instead.


In [22]:
# There are other map tiles available. See https://folium.readthedocs.io/en/latest/quickstart.html
# Instead of Markers we use circles colors are HTML color codes http://htmlcolorcodes.com/
CENTER_US = (39.8333333,-98.585522)
map2 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Stamen Toner')
for row in data.to_records():
    map2.add_child(folium.CircleMarker(location=(row['ApproxLat'],row['ApproxLng']), popup=row['City'], radius=10000, color='#0000FF',  fill_color='#FF3333'))
    
map2

# Choropleths

Choropleths are cartographic overlays based on boundries defined in a geo JSON file.


In [25]:
# State level geo-json overlay choropleth
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_path = state_geojson)
map3


/home/nbuser/anaconda3_420/lib/python3.5/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


TypeError: __init__() missing 1 required positional argument: 'geo_data'

In [14]:
states = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
state_counts = pd.DataFrame( {'Counts' : data['State']. value_counts() } ).sort_index()
state_counts['StateCode'] = state_counts.index
state_data = states.merge(state_counts, how="left", left_on='Abbreviation', right_on='StateCode')
state_data = state_data[['Abbreviation','Counts']]
state_data = state_data.fillna(0)
state_data


,Abbreviation,Counts
0,AL,0.0
1,AK,0.0
2,AZ,0.0
3,AR,0.0
4,CA,10.0
5,CO,0.0
6,CT,0.0
7,DE,0.0
8,DC,0.0
9,FL,1.0


In [15]:
CENTER_US = (39.8333333,-98.585522)
state_geojson = 'WMC3-us-states.json'
map3 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map3.choropleth(geo_path=state_geojson,data=state_data, columns=['Abbreviation','Counts'], 
                key_on ='feature.id', fill_color='BuGn', legend_name='Website Visitors')
map3

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


TypeError: __init__() missing 1 required positional argument: 'geo_data'

In [16]:
# Here's a more straigtforward example with unemployment data:
unemployment = pd.read_csv('https://raw.githubusercontent.com/wrobstory/vincent/master/examples/data/US_Unemployment_Oct2012.csv')
state_geojson = 'WMC3-us-states.json'
map4 = folium.Map(location=CENTER_US, zoom_start=4, tiles=' Open Street Map')
map4.choropleth(geo_path=state_geojson,data=unemployment, 
                columns=['State','Unemployment'], key_on ='feature.id', fill_color='YlOrRd', legend_name='2012 US Unemployment')
map4


/home/nbuser/anaconda3_420/lib/python3.5/site-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


TypeError: __init__() missing 1 required positional argument: 'geo_data'